### Замерим качество Линейной регрессии после обработки данных не просто на отложенной выборке, но и на Кросс-Валидации на 4 фолдах!

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [3]:
processed_data = pd.read_csv('processed_data.csv', index_col='id')
processed_data.shape

(1458644, 5)

#### ! Не перемешивайте данные

In [ ]:

selector = KFold(n_splits=4)

### Шаг №5
### Замерьте качество (MSLE, как и раньше) на Кросс-валидации, 
### используя MSE от log_trip_duration и назначенный selector

### Your code is here
X = processed_data.drop('log_trip_duration', axis=1)
y = processed_data['log_trip_duration']
losses = []

model_1 = LinearRegression()

for train_idx, test_idx in selector.split(X, y):
    X_fold_train, X_fold_test = X.values[train_idx], X.values[test_idx]
    y_fold_train, y_fold_test = y.values[train_idx], y.values[test_idx]

    model_1.fit(X_fold_train, y_fold_train)
    y_fold_pred = model_1.predict(X_fold_test)

    metric = mean_squared_error(y_fold_test, y_fold_pred)
    losses.append(metric)
cross_val_error = np.mean(losses)

print(f"MSLE на Кросс-валидации: {round(cross_val_error, 3)}")

In [ ]:
losses

## Поработал один из хитрых гномов!

В отличие от своих собратьев, третий гном оказался тем еще бездельником в школьные годы, но все равно страстно желал во всем догнать первых двух. И сейчас, желая помочь им в построении модели по предсказанию длительности поездки такси, добавил в данные 20 зашифрованных фичей (их смысл нам не рассказали: какая-то секретная информация о водителях).

Гном думал следующим образом: "Ну не может же модель стать хуже! А тут вот авось и мое нововведение уменьшит ошибку в разы! Тогда и меня станут звать на гномий  data-саммит."

Проверим на кросс-валидации, насколько гном оказался прав!

In [4]:
new_data = pd.read_csv('new_data.csv', index_col='id')

In [5]:
new_data.shape

(1458644, 25)

In [ ]:
### Шаг №6
### Замерьте качество (MSLE, как и раньше) на Кросс-валидации, 
### используя MSE от log_trip_duration и назначенный ранее selector

### Your code is here

# import numpy as np
# import pandas as pd
# from sklearn.model_selection import KFold
# from sklearn.linear_model import LinearRegression
# from sklearn.metrics import mean_squared_error

# selector = KFold(n_splits=4)
# X = new_data.drop('log_trip_duration', axis=1)
# y = new_data['log_trip_duration']
# losses = []

# model_1 = LinearRegression()

# for train_idx, test_idx in selector.split(X, y):
#     X_fold_train, X_fold_test = X.values[train_idx], X.values[test_idx]
#     y_fold_train, y_fold_test = y.values[train_idx], y.values[test_idx]

#     model_1.fit(X_fold_train, y_fold_train)
#     y_fold_pred = model_1.predict(X_fold_test)

#     metric = mean_squared_error(y_fold_test, y_fold_pred)
#     losses.append(metric)
# cross_val_error_2 = np.mean(losses)

# print(f"MSLE: {round(cross_val_error_2, 3)}")

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer

X_new = new_data.drop('log_trip_duration', axis=1)
y_new = new_data['log_trip_duration']

model_2 = LinearRegression()

# Define a custom scorer for mean squared error
mse_scorer = make_scorer(mean_squared_error, greater_is_better=False)

# Perform cross-validation and calculate the negative MSE
cross_val_scores = cross_val_score(model_2, X_new, y_new, cv=4, scoring=mse_scorer)

# Calculate the mean of the negative MSE scores
cross_val_error_2 = -cross_val_scores.mean()

print(f"MSLE: {round(cross_val_error_2, 3)}")

In [ ]:
cross_val_scores

В линейной алгербре зачастую используют понятие **ранга матрицы**. Оно соответствует кол-ву линейно независимых столбцов в матрице. Иными словами, позволяет оценить, есть ли избыток информации в нашем датафрейме. Если ранг матрицы меньше, чем кол-во используемых столбцов, то некоторые фичи следует удалить, ведь иначе возникает ситуация строгой мультиколлинеарности.

Чтобы замерить ранг в наших матрицах объект-признак, можно воспользоваться функцией numpy.linalg.matrix_rank

Константным признаком в данном упражнении можно пренебречь.

In [ ]:
### Создайте переменные rank_processed, rank_new 
### Соответственно равные рангу изначальной матрицы
### с данными и рангу матрицы третьего гнома

### Your code is here
rank_processed = np.linalg.matrix_rank(processed_data.drop('log_trip_duration', axis=1))
rank_new = np.linalg.matrix_rank(new_data.drop('log_trip_duration', axis=1))
print(rank_processed, rank_new)

In [ ]:
### Создайте переменные num_features_processed, num_features_new
### Соответственно равные кол-ву фичей в изначальной матрицы
### с данными и кол-ву фичей у третьего гнома

### Your code is here
num_features_processed = processed_data.drop('log_trip_duration', axis=1).shape[1]
num_features_new = new_data.drop('log_trip_duration', axis=1).shape[1]

In [ ]:
### Шаг №7
print(f"В первой модели всего фичей: {num_features_processed}, - а ранг равен {rank_processed}")

print(f"Во второй модели всего фичей: {num_features_new}, - а ранг равен {rank_new}")

Не кажется ли нам, что из-за новых 20 фичей появилась проблема мультиколлинеарности? Как поступить гному, чтобы, с одной стороны, получить адекватное качество, а с другой стороны, не повредить свое самолюбие и не убирать новые признаки?

Верно! Например, с помощью регуляризации.

Найдите такой параметр регуляризации $\alpha$ для Ridge и Lasso случая, чтобы ошибка MSLE на кросс-валидации оказалась строго меньше 0.4

**ALARM**: используйте процедуру масштабирования данных (воспользуйтесь методом MinMaxScaler) перед тем как применить регуляризацию. Важно - чтобы сохранить концепцию независимости обучения на трейне и на тесте, на каждой итерации кросс-валидации необходимо замерять параметры стандартизации исключительно на трейне, а потом применять на валидационном фолде.

In [ ]:
### Пример, как это можно сделать в цикле
### То есть обучить Lasso, учитывая масштабирование
### Исключительно на трейне на каждой итерации



X = new_data.drop('log_trip_duration', axis=1)
Y = new_data['log_trip_duration']

from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import Lasso

scores = []

for train_index, test_index in selector.split(X):
    
    X_train, X_test = X.values[train_index], X.values[test_index]
    Y_train, Y_test = Y.values[train_index], Y.values[test_index]
    
    ### Фитим исключительно на трейне!
    scaler = MinMaxScaler()
    scaler.fit(X_train)
    
    ### Применяем обученный scaler на трейн и тест
    X_train_scaled = scaler.transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    ### max_iter иногда требуется ставить побольше, 
    ### особенно когда данных много и/или они сложные
    ### этот параметр регулирует верхнюю границу кол-ва
    ### итераций во время обучения
    ### подробнее - в документации
    
    ### По дефолту здесь параметр регуляризации alpha=1
    
    model_lasso = Lasso(max_iter=100000) 
    model_lasso.fit(X_train_scaled, Y_train)
    
    predictions = model_lasso.predict(X_test_scaled)
    
    scores.append(np.mean((predictions - Y_test)**2))

    
print(f"MSLE на Кросс-валидации равен: {np.mean(scores)}")

### P.S. если вы уже умеете работать с конструкциями
### Pipeline, GridSearchCV, cross_validate
### Можете использовать их. Мы познакомимся с ними позже.

In [ ]:
### Теперь найдите оптимальный параметр регуляризации
### для случая Ridge
### Напомним: ошибка на кросс-валидации должно быть 
### строго меньше 0.4

### Шаг №8
### Your code is here
X = new_data.drop('log_trip_duration', axis=1)
Y = new_data['log_trip_duration']

from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import Ridge

scores = []

for train_index, test_index in selector.split(X):
    
    X_train, X_test = X.values[train_index], X.values[test_index]
    Y_train, Y_test = Y.values[train_index], Y.values[test_index]
    
    ### Фитим исключительно на трейне!
    scaler = MinMaxScaler()
    scaler.fit(X_train)
    
    ### Применяем обученный scaler на трейн и тест
    X_train_scaled = scaler.transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    ### max_iter иногда требуется ставить побольше, 
    ### особенно когда данных много и/или они сложные
    ### этот параметр регулирует верхнюю границу кол-ва
    ### итераций во время обучения
    ### подробнее - в документации
    
    ### По дефолту здесь параметр регуляризации alpha=1
    
    model_lasso = Ridge(max_iter=100000, alpha=0.12) 
    model_lasso.fit(X_train_scaled, Y_train)
    
    predictions = model_lasso.predict(X_test_scaled)
    
    scores.append(np.mean((predictions - Y_test)**2))

    
print(f"MSLE на Кросс-валидации равен: {np.mean(scores)}")


In [19]:
### Найдите оптимальный параметр регуляризации
### для случая Lasso
### Напомним: ошибка на кросс-валидации должно быть 
### строго меньше 0.4

### Шаг №9
### Your code is here

X = new_data.drop('log_trip_duration', axis=1)
Y = new_data['log_trip_duration']

from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import Lasso

scores = []
selector = KFold(n_splits=4)

for train_index, test_index in selector.split(X):
    
    X_train, X_test = X.values[train_index], X.values[test_index]
    Y_train, Y_test = Y.values[train_index], Y.values[test_index]
    
    scaler = MinMaxScaler()
    scaler.fit(X_train)
    
    X_train_scaled = scaler.transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    model_lasso = Lasso(max_iter=100000, alpha=0.00001) 

    model_lasso.fit(X_train_scaled, Y_train)
    
    predictions = model_lasso.predict(X_test_scaled)
    
    scores.append(np.mean((predictions - Y_test)**2))

    
print(f"MSLE на Кросс-валидации равен: {np.mean(scores)}")

MSLE на Кросс-валидации равен: 0.3984816859563738


In [17]:
scores

[0.47021966223477824,
 0.5069334082998875,
 0.5087189035231492,
 0.5081503244152612]